#### Configuración

In [116]:
## Celda de configuración

# Librerias
import math
import time
import sys
import os

# Instancias y algoritmos
sys.path.append(os.path.abspath("../Instances"))
from InsInventory import generar_datos, visualizar, plot_plan_produccion

# Programación Dinámica
sys.path.append(os.path.abspath("../DP/Env"))         # entorno del problema
sys.path.append(os.path.abspath("../DP/Algorithms/")) # algoritmos RL / DP
sys.path.append(os.path.abspath("../DP/Visual/"))     # visualización de políticas / valores

from Inventory import InventoryEnv                    # Clase que define el entorno tipo iventario

from value_iteration import value_iteration           # iteración de valores
from policy_evaluation import policy_evaluation       # evaluación de políticas
from policy_iteration import policy_iteration         # iteración de políticas

from value_states import value_states_visual          # visualización de V(s)
from policy_dag import draw_policy_dag                # grafo de política óptima

# Modelos de Producción
sys.path.append(os.path.abspath("../Producción"))

from EOQ import heuristica_eoq                        # EOQ
from sS import heuristica_ss                          # sS
from Silver_Meal import heuristica_silver_meal        # Silver Meal

# OptiCoffee

El Área de Producción de **OptiCoffee** lo ha contactado para realizar la *planeación de la producción de toneladas de café para los próximos 'n' meses*. El Área de Producción debe decidir *cuántas toneladas de café producir mensualmente* con el fin de *satisfacer la demanda al final de cada mes*.

Actualmente, OptiCoffee cuenta con un *inventario de 'n' toneladas* de café almacenadas en su bodega. Al final de cada mes, el café que no haya sido utilizado debe ser *almacenado en inventario* y puede ser utilizado para satisfacer la demanda de los siguientes meses.

La empresa le pone a su disposición los *costos mensuales de producción y almacenamiento* de una tonelada de café, así como la *demanda mensual* de toneladas de café que tiene que satisfacer OptiCoffee. Es importante tener en cuenta que los **costos y la demanda varían dependiendo del mes**, estos son a modo de ejemplo.

Naturalmente, el Área de Producción de OptiCoffee desea *minimizar sus costos totales a lo largo del horizonte de planeación* (i.e., 12 meses).

In [117]:
# 1. TODO Defina el número de periodos a usar.
NUM_PERIODOS = 10

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Generar datos.
horizonte_planeacion = generar_datos(num_periodos = NUM_PERIODOS)
visualizar(horizonte_planeacion)

# . Guardar resultados.
resultados = []

## Optimización (MIP)

#### Conjuntos

$$
M \quad \text{: Conjunto de meses}
$$

#### Parámetros

$$
c_t \quad \text{: Costo de producir una tonelada de café en el mes } t \in M
$$

$$
h_t \quad \text{: Costo de almacenar una tonelada de café en el mes } t \in M
$$

$$
d_t \quad \text{: Demanda de café en el mes } t \in M
$$

$$
I^0 \quad \text{: Inventario inicial (toneladas)}
$$


#### Variables de decisión

$$
x_t \quad \text{: Toneladas de café a producir en el mes } t \in M
$$

$$
I_t \quad \text{: Inventario de toneladas de café al final del mes } t \in M
$$


#### Modelo en Palabras

**Minimizar:**

> Costos de producción + Costos de almacenamiento

**Sujeto a:**

* Modelar el inventario para el primer mes
* Modelar el inventario para los demás meses
* Naturaleza de las variables

#### Modelo Matemático

Minimizar:

$$
\sum_{t \in M} (c_t x_t + h_t I_t) \tag{1}
$$

Sujeto a:

Inventario en el primer mes:

$$
I_1 = I^0 + x_1 - d_1 \tag{2}
$$

Inventario para los meses siguientes:

$$
I_t = I_{t-1} + x_t - d_t, \quad \forall t \in M \mid t > 1 \tag{3}
$$

Naturaleza de las variables:

$$
x_t, I_t \geq 0, \quad \forall t \in M \tag{4}
$$

#### Notas

* (1) minimiza los costos de producción más los costos de almacenamiento.
* (2) modela el inventario para el primer mes.
* (3) modela el inventario para los demás meses.
* (4) describe la naturaleza de las variables.

In [118]:
# 2. TODO Extraiga los parámetros relevantes desde el DataFrame

# M: períodos del horizonte.
M = list(horizonte_planeacion.index)

# c_t: costo de compra/producción por período
c = dict(horizonte_planeacion['c_t'].squeeze())

# h_t: costo de mantenimiento (holding) por período
h = dict(horizonte_planeacion['h_t'].squeeze())

# d_t: demanda esperada por período
d = dict(horizonte_planeacion['d_t'].squeeze())

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# I_0: Inventario inicial
I_0 = int(min(horizonte_planeacion['d_t']) * 0.25)

In [119]:
# 3. TODO Complete la siguiente función

import pulp as lp

def optimizacion_inventario(M, c, h, d, I_0=0):

    # Definir el problema.
    model = lp.LpProblem("OptiCoffi", lp.LpMinimize)

    # Variables de decisión
    x = {t: lp.LpVariable(f'x_{t}', lowBound=0, cat=lp.LpInteger) for t in M}
    I = {t: lp.LpVariable(f'I_{t}', lowBound=0, cat=lp.LpInteger) for t in M}
    
    # Función Objetivo
    model += lp.lpSum(c[t] * x[t] + h[t] * I[t] for t in M)

    # Restricciones
    # Balance inventario
    for t in M:
        if t == 1:
            model += I[1] == I_0 + x[1] - d[1]  # usa inventario inicial
        else:
            model += I[t] == I[t-1] + x[t] - d[t]
    

    # ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.
    
    # Resolver
    solver = lp.getSolver('PULP_CBC_CMD', msg=False)
    model.solve(solver)

    # Reportar
    print(f'El optimizador llegó a una solución: {lp.LpStatus[model.status]}.')
    return model, x, I


In [120]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr el optimizador.
t0 = time.perf_counter()
model, x, I = optimizacion_inventario(M, c, h, d, I_0)
elapsed = time.perf_counter() - t0

# . Extarer resultados
if lp.LpStatus[model.status] == 'Optimal':

    obj_lp = round(lp.value(model.objective), 3)                        #   ▸ Función Objetivo
    produccion = {t: x[t].varValue for t in M}                          #   ▸ Producción
    inventario = {1: I_0, **{t + 1: I[t].varValue for t in M}}          #   ▸ Almacenado
    cost_produccion = round(sum(x[t].varValue * c[t] for t in M), 3)    #   ▸ Costo total de producción
    cost_inventario = round(sum(I[t].varValue * h[t] for t in M), 3)    #   ▸ Costo total de almacenar

    # 5. Reporte
    print(f'FO (valor total): {cost_produccion} (Producción) + {cost_inventario} (Inventario) = {obj_lp}')
    print(f'Cantidad de toneladas pedidas (valor total): {sum(produccion.values())}.')
    print(f'Cantidad de toneladas en inventario (valor total): {sum(inventario.values())}.')
    plot_plan_produccion(M, d, produccion, inventario, cost_produccion, cost_inventario, obj_lp)

    # 6. Registrar resultados
    resultados.append({
        "Método"             : "LP",
        "Costo Total"        : obj_lp,
        "Costo producir"     : cost_produccion,
        "Costo inventario"   : cost_inventario,
        "Ton pedidas (total)": sum(produccion.values()),
        "Inventario(total)"  : sum(inventario.values()),
        "Tiempo (s)"         : elapsed
    })

else:
    print(f"ERROR: Algo quedo mal con el modelo de optimización: {lp.LpStatus[model.status]}")


El optimizador llegó a una solución: Optimal.
FO (valor total): 355.23 (Producción) + 15.24 (Inventario) = 370.47
Cantidad de toneladas pedidas (valor total): 51.0.
Cantidad de toneladas en inventario (valor total): 7.0.


## Programación Dinámica

In [121]:
# 4. TODO Mapee cada uno de los siguientes datos usados por el ambiente.

demand         = list(d.values())  # demanda por período
cost_producir  = list(c.values())  # costo de producción/compra por período
cost_almacenar = list(h.values())  # costo de mantenimiento por período

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Definir la capacidad del ambiente, y el inventario inicial
capacity        = int(sum(demand) / 2)  # capacidad (heurística: 50% de la demanda total)
start_inventory = I_0                   # inventario inicial

# . Crear el ambiente
env = InventoryEnv(demand, cost_producir, cost_almacenar, capacity, start_inventory)
print(env)

InventoryEnv(Horizonte = 10, Capacidad = 26, #_Estados = 297)


### Policy Evaluation

```text
# Calcula Vπ para una política fija π
# Entradas:
#   S          : conjunto de estados
#   π(s)       : política dada (acción para cada s)
#   p(s',r|s,a): modelo de transición y recompensa
#   γ          : factor de descuento (0 ≤ γ ≤ 1)
#   θ          : umbral de convergencia
# Salida:
#   Vπ(s)      : valor esperado para cada estado bajo π
# -----------------------------------------------------

# Inicialización
V(s) ← 0  para todo s ∈ S

# Evaluación iterativa
repetir
    Δ ← 0
    para cada estado s ∈ S:
        v ← V(s)                                    # valor anterior
        a ← π(s)                                    # acción dictada por la política
        V(s) ← Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]
        Δ ← max(Δ, |v − V(s)|)
hasta que Δ < θ

return V
```

In [122]:
# 5. TODO DEFINIR POLÍTICA: Escriba la siguiente política:
# Política: ordenar justo lo faltante para cubrir demanda en t
#   key = (t, S) donde S es inventario disponible; acción = max(0, demanda - S)

propose_policy = {(t, S): max(0, env.demand[t] - S) for t in range(env.n) for S in range(env.capacity + 1)}

# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

policy = propose_policy.copy()

In [123]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
t0 = time.perf_counter()
V = policy_evaluation(env, propose_policy)
elapsed = time.perf_counter() - t0

# . Reportar
obj_dp, costos_prod, costos_inv, produccion, inventario, trayecto = env.report_from_policy(policy)
plot_plan_produccion(M, d, produccion, inventario, sum(costos_prod), sum(costos_inv), obj_dp)

# . Almacenar resultados
resultados.append({
    "Método"             : "Policy evaluation",
    "Costo Total"        : obj_dp,
    "Costo producir"     : sum(costos_prod),
    "Costo inventario"   : sum(costos_inv),
    "Ton pedidas (total)": sum(produccion),
    "Inventario(total)"  : sum(inventario),
    "Tiempo (s)"         : elapsed
})

FO (valor total): 374.61 (Producción) + 0.0 (Inventario) = 374.61
Cantidad de toneladas pedidas (valor total): 55.
Cantidad de toneladas en inventario (valor total): 55.


### Policy Iteration

```text
# Alterna evaluación y mejora hasta que la política se estabiliza
# Entradas:
#   S, A(s), p(s',r|s,a), γ, θ
# Salidas:
#   π*  : política óptima
#   V*  : función de valor óptima
# -------------------------------------------

# 1. Inicialización
π(s) ← acción cualquiera  para todo s ∈ S
V(s) ← 0

bucle:
    # 2. Evaluar la política actual π  (igual que el bloque anterior)
    repetir
        Δ ← 0
        para cada s ∈ S:
            v ← V(s)
            a ← π(s)
            V(s) ← Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]
            Δ ← max(Δ, |v − V(s)|)
    hasta que Δ < θ

    # 3. Mejorar la política
    policy_stable ← true
    para cada s ∈ S:
        old ← π(s)
        π(s) ← argmax_a Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]
        si old ≠ π(s):
            policy_stable ← false

    si policy_stable:
        break

return π, V

```

In [124]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
t0 = time.perf_counter()
star_policy, V_star = policy_iteration(env, policy)
elapsed = time.perf_counter() - t0

# . Reportar
obj_dp, costos_prod, costos_inv, produccion, inventario, trayecto = env.report_from_policy(star_policy)
plot_plan_produccion(M, d, produccion, inventario, sum(costos_prod), sum(costos_inv), obj_dp)

# . Almacenar resultados
resultados.append({
    "Método"             : "Policy iteration",
    "Costo Total"        : obj_dp,
    "Costo producir"     : sum(costos_prod),
    "Costo inventario"   : sum(costos_inv),
    "Ton pedidas (total)": sum(produccion),
    "Inventario(total)"  : sum(inventario),
    "Tiempo (s)"         : elapsed
})


FO (valor total): 355.23 (Producción) + 15.24 (Inventario) = 370.47
Cantidad de toneladas pedidas (valor total): 55.
Cantidad de toneladas en inventario (valor total): 55.


### Value Iteration

```text
# Aproxima V* directamente y luego deriva π*
# Entradas:
#   S, A(s), p(s',r|s,a), γ, θ
# Salidas:
#   π* : política óptima
#   V* : función de valor óptima
# -------------------------------------------

# Fase 1: aproximar V*
V(s) ← 0  para todo s ∈ S

repetir
    Δ ← 0
    para cada s ∈ S:
        v ← V(s)
        V(s) ← max_a Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]
        Δ ← max(Δ, |v − V(s)|)
hasta que Δ < θ

# Fase 2: derivar π* usando V*
para cada s ∈ S:
    π*(s) ← argmax_a Σ_{s',r} p(s',r | s,a) · [ r + γ · V(s') ]

return π*, V
```

In [125]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
t0 = time.perf_counter()
opt_policy, V_opt = value_iteration(env)
elapsed = time.perf_counter() - t0

# . Reportar
obj_dp, costos_prod, costos_inv, produccion, inventario, trayecto = env.report_from_policy(opt_policy)
plot_plan_produccion(M, d, produccion, inventario, sum(costos_prod), sum(costos_inv), obj_dp)

# . Almacenar resultados
resultados.append({
    "Método"             : "Value iteration",
    "Costo Total"        : obj_dp,
    "Costo producir"     : sum(costos_prod),
    "Costo inventario"   : sum(costos_inv),
    "Ton pedidas (total)": sum(produccion),
    "Inventario(total)"  : sum(inventario),
    "Tiempo (s)"         : elapsed
})

FO (valor total): 355.23 (Producción) + 15.24 (Inventario) = 370.47
Cantidad de toneladas pedidas (valor total): 55.
Cantidad de toneladas en inventario (valor total): 55.


## Heurísticas

| **Heurística** | **¿Qué hace? (explicación sencilla)** | **Pasos básicos (visión “receta”)** |
|---------------|----------------------------------------|--------------------------------------|
| **EOQ por bloques de períodos** | Divide el horizonte en “paquetes” de meses. Para cada paquete se fabrica todo de una vez de modo que la demanda acumulada sea parecida al lote $EOQ$. | 1. Inicia en el mes actual.<br>2. Suma demanda futura hasta alcanzar aproximadamente $EOQ$.<br>3. Produce al inicio del paquete la cantidad faltante.<br>4. Atiende cada mes con ese inventario, corrigiendo si quedas corto.<br>5. Avanza al siguiente mes no cubierto y repite. |
| **Política $(s,\,S)$** | Coloca un punto de alarma $s$. Si el inventario al empezar el mes está por debajo de $s$, lo rellenas hasta $S$; si no, no produces. Nunca se permiten faltantes. | 1. Observa inventario inicial.<br>2. Si $I < s$, produce $S - I$.<br>3. Si aún no cubres la demanda del mes, produce el extra necesario.<br>4. Resta la demanda.<br>5. Si el inventario quedó negativo, añade esa cantidad a la producción y fija $I = 0$. |
| **Silver‑Meal** | Pregunta: “¿Conviene cubrir 1, 2, 3… meses con este pedido?” Calcula el costo promedio por mes; mientras baje o se mantenga, extiende el bloque. Cuando sube, se detiene. | 1. Colócate en el primer mes sin cubrir.<br>2. Extiende el bloque mes a mes; en cada paso calcula el costo promedio $(K + \text{holding})/\#\text{meses}$.<br>3. Guarda la última extensión que no aumentó el promedio.<br>4. Produce al inicio del bloque lo necesario.<br>5. Consume mes a mes; si falta inventario, ajusta la producción inicial.<br>6. Salta al siguiente mes sin cubrir y repite. |


### EOQ

In [126]:
# 6. TODO Defina el criterio EOQ.
D_total = sum(d[t] for t in M)  / len(M)       # demanda promedio
H_prom  = sum(h[t] for t in M) / len(M)        # holding promedio
C_prom  = sum(c[t] for t in M) / len(M)        # producción promedio
EOQ = math.sqrt(2 * D_total * C_prom / H_prom) # fórmula EOQ clásica


# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
start = time.perf_counter()
produccion, inventario, inventario_fin = heuristica_eoq(M, d, I_0, EOQ)
end = time.perf_counter()

# . Reportar
cost_produccion = sum(produccion[t] * c[t] for t in M)
cost_inventario = sum(inventario_fin[t] * h[t] for t in M)
obj_hh = cost_produccion + cost_inventario  # recalculado aquí

print(f'FO (valor total): {cost_produccion:.2f} (Producción) + {cost_inventario:.2f} (Inventario) = {obj_hh:.2f}')
print(f'Cantidad de toneladas producidas (valor total): {sum(produccion.values())}.')
print(f'Cantidad de toneladas en inventario (valor total): {sum(inventario_fin.values())}.')

plot_plan_produccion(M, d, produccion, inventario, cost_produccion, cost_inventario, obj_hh)      #   ▸ Reportar

# . Almacenar resultados
resultados.append({
    "Método"             : "EOQ",
    "Costo Total"        : obj_hh,
    "Costo producir"     : cost_produccion,
    "Costo inventario"   : cost_inventario,
    "Ton pedidas (total)": sum(produccion.values()),
    "Inventario(total)"  : sum(inventario_fin.values()),
    "Tiempo (s)"         : end - start
})


FO (valor total): 351.47 (Producción) + 33.00 (Inventario) = 384.47
Cantidad de toneladas producidas (valor total): 51.
Cantidad de toneladas en inventario (valor total): 10.


### sS

In [127]:
# 7. TODO Defina los umbrales.
s = max(d.values())          # umbral de disparo: si inv < s, se repone
S = 1.5 * min(d.values())    # nivel objetivo tras reponer (ojo: puede quedar S < s)


# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
start = time.perf_counter()
produccion, inventario, inventario_fin = heuristica_ss(M, d, I_0, s, S)
end = time.perf_counter()

# . Reportar
cost_produccion = sum(produccion[t] * c[t] for t in M)
cost_inventario = sum(inventario_fin[t] * h[t] for t in M)
obj_hh = cost_produccion + cost_inventario  # recalculado aquí

print(f'FO (valor total): {cost_produccion:.2f} (Producción) + {cost_inventario:.2f} (Inventario) = {obj_hh:.2f}')
print(f'Cantidad de toneladas producidas (valor total): {sum(produccion.values())}.')
print(f'Cantidad de toneladas en inventario (valor total): {sum(inventario_fin.values())}.')

plot_plan_produccion(M, d, produccion, inventario, cost_produccion, cost_inventario, obj_hh)      #   ▸ Reportar

# . Almacenar resultados
resultados.append({
    "Método"             : "(s,S)",
    "Costo Total"        : obj_hh,
    "Costo producir"     : cost_produccion,
    "Costo inventario"   : cost_inventario,
    "Ton pedidas (total)": sum(produccion.values()),
    "Inventario(total)"  : sum(inventario.values()),
    "Tiempo (s)"         : end - start
})


FO (valor total): 370.58 (Producción) + 34.38 (Inventario) = 404.96
Cantidad de toneladas producidas (valor total): 52.
Cantidad de toneladas en inventario (valor total): 9.


### Silver Meal

In [128]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# . Correr
start = time.perf_counter()
produccion, inventario, inventario_fin = heuristica_silver_meal(M, c, h, d, I_0)
end = time.perf_counter()

# . Reportar
cost_produccion = sum(produccion[t] * c[t] for t in M)
cost_inventario = sum(inventario_fin[t] * h[t] for t in M)
obj_hh = cost_produccion + cost_inventario  # recalculado aquí

print(f'FO (valor total): {cost_produccion:.2f} (Producción) + {cost_inventario:.2f} (Inventario) = {obj_hh:.2f}')
print(f'Cantidad de toneladas producidas (valor total): {sum(produccion.values())}.')
print(f'Cantidad de toneladas en inventario (valor total): {sum(inventario_fin.values())}.')

plot_plan_produccion(M, d, produccion, inventario, cost_produccion, cost_inventario, obj_hh)      #   ▸ Reportar

# . Almacenar resultados
resultados.append({
    "Método"             : "Silver-Meal",
    "Costo Total"        : obj_hh,
    "Costo producir"     : cost_produccion,
    "Costo inventario"   : cost_inventario,
    "Ton pedidas (total)": sum(produccion.values()),
    "Inventario(total)"  : sum(inventario_fin.values()),
    "Tiempo (s)"         : end - start
})


FO (valor total): 338.21 (Producción) + 107.72 (Inventario) = 445.93
Cantidad de toneladas producidas (valor total): 51.
Cantidad de toneladas en inventario (valor total): 27.


## Resultados

In [129]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

import pandas as pd

# -- Crear DataFrame desde la lista de resultados --
df_resultados = pd.DataFrame(resultados)

# -- Establecer 'Método' como índice --
df_resultados.set_index("Método", inplace=True)

# -- Ordenar por empleos generados (de mayor a menor) --
df_resultados.sort_values("Costo Total", ascending=True, inplace=True)

# -- Visualizar resultados ordenados --
df_resultados

,Costo Total,Costo producir,Costo inventario,Ton pedidas (total),Inventario(total),Tiempo (s)
Método,,,,,,
LP,370.47,355.23,15.24,51.0,7.0,0.039233
Policy iteration,370.47,355.23,15.24,55.0,55.0,0.005307
Value iteration,370.47,355.23,15.24,55.0,55.0,0.028383
Policy evaluation,374.61,374.61,0.00,55.0,55.0,0.001436
EOQ,384.47,351.47,33.00,51.0,10.0,0.000050
"(s,S)",404.96,370.58,34.38,52.0,9.0,0.000033
Silver-Meal,445.93,338.21,107.72,51.0,27.0,0.000047


In [130]:
# -- Tomar la solución base de referencia (LP) --
base_lp = df_resultados.loc["LP"]

# -- Calcular el GAP porcentual relativo al método LP (excepto para Tiempo) --
gap_df = ((df_resultados - base_lp) / base_lp) * 100

# -- Reemplazar la columna de tiempo con "x veces LP" en lugar de porcentaje --
tiempo_ratio = (df_resultados["Tiempo (s)"] / base_lp["Tiempo (s)"]).round(4)

# -- Redondear GAPs a 3 decimales --
gap_df = gap_df.round(3)

# -- Renombrar columnas GAP --
gap_df.columns = [col + " GAP (%)" for col in gap_df.columns]

# -- Reemplazar columna de tiempo con "x veces LP" --
gap_df["Tiempo (s) (x veces LP)"] = tiempo_ratio

# -- Eliminar columna anterior de GAP de tiempo --
gap_df = gap_df.drop(columns=["Tiempo (s) GAP (%)"])

# -- Mostrar tabla --
gap_df

,Costo Total GAP (%),Costo producir GAP (%),Costo inventario GAP (%),Ton pedidas (total) GAP (%),Inventario(total) GAP (%),Tiempo (s) (x veces LP)
Método,,,,,,
LP,0.000,0.000,0.000,0.000,0.000,1.0000
Policy iteration,0.000,0.000,0.000,7.843,685.714,0.1353
Value iteration,0.000,0.000,0.000,7.843,685.714,0.7235
Policy evaluation,1.117,5.456,-100.000,7.843,685.714,0.0366
EOQ,3.779,-1.058,116.535,0.000,42.857,0.0013
"(s,S)",9.310,4.321,125.591,1.961,28.571,0.0008
Silver-Meal,20.369,-4.791,606.824,0.000,285.714,0.0012


#### Aux

In [131]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

def plot_costos_barras(df):
    """
    Muestra tres métricas económicas en barras: Costo Total, Costo producir,
    Costo inventario.  Cada barra lleva el nombre del método vertical dentro.
    """
    metricas = [
        "Costo Total",
        "Costo producir",
        "Costo inventario"
    ]
    titulos = [
        "Costo total ($)",
        "Costo de producir ($)",
        "Costo de inventario ($)"
    ]
    colores = ["#4caf50", "#2196f3", "#ff9800"]      # verde, azul, naranja
    metodos = df.index                               # índice = nombre de cada método
    
    fig = make_subplots(rows=1, cols=3,
                        subplot_titles=titulos,
                        horizontal_spacing=0.10)

    for i, (col, color) in enumerate(zip(metricas, colores), start=1):
        valores = df[col]
        fig.add_trace(
            go.Bar(
                x=metodos,
                y=valores,
                text=metodos,                # texto = nombre del método
                textposition="inside",
                textangle=-90,
                insidetextanchor="middle",
                marker_color=color,
                textfont=dict(color="white", size=12, weight="bold")
            ),
            row=1, col=i
        )
        fig.update_yaxes(title_text=col, row=1, col=i)

    fig.update_layout(
        showlegend=False,
        plot_bgcolor="white",
        paper_bgcolor="white",
    )
    fig.show()

In [132]:
def plot_eficiencia(df):
    """
    Dispersión log‑log: eje X = Tiempo de ejecución (s),
                       eje Y = Costo Total.
    Cada punto es un método con color propio y grid activado.
    """
    df_plot = df.reset_index().rename(columns={"index": "Método"})
    
    fig = px.scatter(
        df_plot,
        x="Tiempo (s)",
        y="Costo Total",
        color="Método",
        size_max=15
    )

    fig.update_traces(
        marker=dict(size=12, line=dict(width=1, color="black"))
    )

    fig.update_xaxes(type="log",
                     showgrid=True, gridcolor="lightgray", zeroline=False)
    fig.update_yaxes(showgrid=True, gridcolor="lightgray", zeroline=False)

    fig.update_layout(
        title="Eficiencia: Costo total vs Tiempo",
        xaxis_title="Tiempo de ejecución (s)  [escala log]",
        yaxis_title="Costo total ($)",
        plot_bgcolor="white",
        paper_bgcolor="white",
        legend_title="Método",
    )
    fig.show()

In [133]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_gaps(gap_df):
    """
    Muestra tres GAPs porcentuales en barras verticales:
      ▸ Costo Total   ▸ Costo de producir   ▸ Costo de inventario
    Cada subplot tiene las barras de todos los métodos con la línea 0 % como referencia.
    """
    metricas = [
        "Costo Total GAP (%)",
        "Costo producir GAP (%)",
        "Costo inventario GAP (%)"
    ]
    titulos = [
        "GAP en costo total (%)",
        "GAP en costo producir (%)",
        "GAP en costo inventario (%)"
    ]
    colores = ["#4caf50", "#2196f3", "#ff9800"]
    metodos = gap_df.index

    fig = make_subplots(rows=1, cols=3, subplot_titles=titulos, horizontal_spacing=0.10)

    for i, (col, color) in enumerate(zip(metricas, colores), start=1):
        valores = gap_df[col]

        fig.add_trace(
            go.Bar(
                x=metodos,
                y=valores,
                marker_color=color,
                text=[f"{v:.1f} %" for v in valores],
                textposition="outside",
                showlegend=False
            ),
            row=1, col=i
        )

        fig.add_hline(y=0, line_color="black", line_width=1, row=1, col=i)
        fig.update_yaxes(title_text="Porcentaje", row=1, col=i)

    fig.update_layout(
        plot_bgcolor="white",
        paper_bgcolor="white",
        title_text="Comparación de GAPs (costos)",
        title_x=0.5,
    )
    fig.show()


In [134]:
def plot_tiempo_relativo(gap_df, ordenar=True):
    """
    Barras horizontales: Tiempo (× LP) para cada método.
    • LP en gris, otros en azul.
    • Etiqueta del valor a la derecha.
    • Si ordenar=True se listan de mayor a menor.
    """
    data = (gap_df.sort_values("Tiempo (s) (x veces LP)", ascending=False)
            if ordenar else gap_df)

    colores = ["#888888" if m == "LP" else "#2196f3" for m in data.index]
    max_val = data["Tiempo (s) (x veces LP)"].max()
    limite_x = max_val * 1.2

    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=data["Tiempo (s) (x veces LP)"],
        y=data.index,
        orientation="h",
        marker_color=colores,
        text=[f"{v:.3f}" for v in data["Tiempo (s) (x veces LP)"]],
        textposition="outside"
    ))

    fig.update_layout(
        xaxis=dict(title="Tiempo relativo (× LP)", range=[0, limite_x]),
        title="Velocidad relativa a LP",
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=False
    )
    fig.show()

#### Comparar

In [135]:
plot_costos_barras(df_resultados)
plot_eficiencia(df_resultados)

In [136]:
plot_gaps(gap_df)
plot_tiempo_relativo(gap_df)

## (Opcional) Visualizar PD

In [137]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# Visualización  de V(s)
if env.capacity <= 20:

    # Evaluación de política
    print("Policy Evaluation")
    value_states_visual(env, V, propose_policy)

    # Política óptima vía iteración de políticas
    print("Policy Iteration")
    value_states_visual(env, V_star, star_policy)

    # Política óptima vía iteración de valores
    print("Value Iteration")
    value_states_visual(env, V_opt, opt_policy)

else:
    print("Para visualizar, use una instancia más pequeña.")

Para visualizar, use una instancia más pequeña.


In [138]:
# ----- DESDE ACÁ EN ADELANTE, NO TOQUE EL CÓDIGO DE ESTÁ CELDA.

# Visualización de las decisiones para cada política evaluada
if env.capacity <= 20:

    # Evaluación de política
    print("Policy Evaluation")
    draw_policy_dag(env, propose_policy, initial_state=(0, env.capacity))
    print()

    # Política óptima vía iteración de políticas
    print("Policy Iteration")
    draw_policy_dag(env, star_policy, initial_state=(0, env.capacity))
    print()

    # Política óptima vía iteración de valores
    print("Value Iteration")
    draw_policy_dag(env, opt_policy, initial_state=(0, env.capacity))
    print()

else:
    print("Para visualizar, use una instancia más pequeña.")


Para visualizar, use una instancia más pequeña.
